In [1]:
import cv2
import numpy as np
import random

class MedicalImageProcessor:
    def __init__(self, output_path):
        # self.image = cv2.imread(image_path)
        self.output_path = output_path

    # RGB to Grey Scale
    def rgb_to_gray(self, image_path):
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        output_path = f'{self.output_path}/gray_scale.jpg'
        cv2.imwrite(output_path, image)

        return image


    # Rotation (-45, 45, 90, 180, 270)
    def rotate(self, image_path, angle):
        image = cv2.imread(image_path)
        if len(image.shape) == 3:
            shape_tuple = image.shape
            rows, cols = shape_tuple[0], shape_tuple[1]
            rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
            image = cv2.warpAffine(image, rotation_matrix, (cols, rows))

            cv2.imwrite(f'{self.output_path}/rotated_{str(angle)}.jpg', image)
            return image
    
    # Flip (Vertical + Horizontal)
    def flip(self, image_path, vertical=True, horizontal=True):
        image = cv2.imread(image_path)
        if vertical:
            image = cv2.flip(image, 0)

            output_path = f"{self.output_path}/flipped_vertical.jpg"
            cv2.imwrite(output_path, image)
            return output_path
        
        if horizontal:
            image = cv2.flip(image, 1)
            output_path = f"{self.output_path}/flipped_horizontal.jpg"
            cv2.imwrite(output_path, image)
            return output_path

    # Cropping
    def crop(self, image_path, x, y, width, height):
        """
        x: This parameter specifies the x-coordinate (horizontal position) of the top-left corner of the cropping region. 
        It determines where the cropping operation will start horizontally.

        y: This parameter specifies the y-coordinate (vertical position) of the top-left corner of the cropping region. 
        It determines where the cropping operation will start vertically.

        width: This parameter specifies the width of the cropping region. 
        It determines how wide the cropped region will be in pixels, starting from the x coordinate.

        height: This parameter specifies the height of the cropping region. 
        It determines how tall the cropped region will be in pixels, starting from the y coordinate.
        """

        image = cv2.imread(image_path)
        image = image[y:y+height, x:x+width]
        output_path = f"{self.output_path}/crop_{x}_{y}_{width}_{height}.jpg"
        cv2.imwrite(output_path, image)
        return output_path


    # Gaussian Noise is a statistical noise with a Gaussian (normal) distribution
    def add_gaussian_noise(self, image_path, mean=0, std=25):

        image = cv2.imread(image_path)
        noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
        image = cv2.add(image, noise)

        output_path = f"{self.output_path}/g_noise_{mean}_{std}.jpg"
        cv2.imwrite(output_path, image)
        return output_path

    # Shear is a geometric augmentation that changes a form of an image along a specific axis to create a different perception angle   
    def shear(self,image_path, shear_x=0, shear_y=0):
        image = cv2.imread(image_path)
        shear_matrix = np.float32([[1, shear_x, 0], [shear_y, 1, 0]])
        image = cv2.warpAffine(image, shear_matrix, (image.shape[1], image.shape[0]))

        output_path = f"{self.output_path}/shear_{shear_x}_{shear_y}.jpg"
        cv2.imwrite(output_path, image)
        return output_path


    # Change in Image Contrast
    def adjust_contrast(self, image_path, alpha=1.5, beta=0):

        image = cv2.imread(image_path)
        image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

        output_path = f"{self.output_path}/contrast_{alpha}_{beta}.jpg"
        cv2.imwrite(output_path, image)
        return output_path

    # Scaling
    def scale(self, image_path, scale_factor):
        image = cv2.imread(image_path)
        image = cv2.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
        output_path = f"{self.output_path}/scale_{scale_factor}.jpg"
        cv2.imwrite(output_path, image)
        return output_path

    # Channel Shuffling
    def channel_shuffle(self, image_path):
        image = cv2.imread(image_path)
        channels = cv2.split(image)
        random.shuffle(channels)
        image = cv2.merge(channels)
        output_path = f"{self.output_path}/channel_shuffle.jpg"
        cv2.imwrite(output_path, image)
        return output_path
    


    # Heatmap + Heatmap to greyscale
    def heatmap_to_grayscale(self, image_path):
        image = cv2.imread(image_path)
        image = cv2.applyColorMap(image, cv2.COLORMAP_JET)
        output_path_rgb = f"{self.output_path}/heatmap_rgb.jpg"
        cv2.imwrite(output_path_rgb, image)

        image = image.rgb_to_gray()
        output_path_gray = f"{self.output_path}/heatmap_rgb.jpg"
        cv2.imwrite(output_path_gray, image)

    # Driver
    def apply_transformations(self, transformations):
        for transform in transformations:
            if transform == 'rgb_to_gray':
                self.rgb_to_gray()
            elif transform.startswith('rotate_'):
                angle = int(transform.split('_')[1])
                self.rotate(angle)
            elif transform == 'flip':
                self.flip()
            elif transform == 'crop':
                x, y, width, height = random.randint(0, 100), random.randint(0, 100), random.randint(100, 200), random.randint(100, 200)
                self.crop(x, y, width, height)
            elif transform == 'add_gaussian_noise':
                self.add_gaussian_noise()
            elif transform.startswith('shear_'):
                shear_angle = int(transform.split('_')[1])
                self.shear(shear_angle)
            elif transform == 'adjust_contrast':
                self.adjust_contrast()
            elif transform == 'scale':
                scale_factor = random.uniform(0.5, 2.0)
                self.scale(scale_factor)
            elif transform == 'channel_shuffle':
                self.channel_shuffle()
            elif transform == 'heatmap_to_grayscale':
                self.heatmap_to_grayscale()

In [2]:
image_path = 'medical images/brain_tumor.jpg'
output_dir = "output_dir"
# transformations = ['rgb_to_gray', 'rotate_45', 'rotate_-45', 'rotate_90', 'rotate_180', 'rotate_270', 'flip', 'crop', 'add_gaussian_noise', 'shear_45', 'shear_-45', 'adjust_contrast', 'scale', 'channel_shuffle', 'heatmap_to_grayscale']

instance = MedicalImageProcessor(output_path= output_dir)

# instance_gray = instance.rgb_to_gray(image_path)

In [3]:
image_path = "output_dir/gray_scale.jpg"
# instance_rotated = instance.rotate(image_path = image_path, angle = 180)

In [4]:
# instance_flipped = instance.flip(image_path= image_path, vertical=True, horizontal=False)

In [5]:
# # x, y, width, height = random.randint(0, 100), random.randint(0, 100), random.randint(100, 200), random.randint(100, 200)
# instance_cropped = instance.crop(image_path= image_path, x= 300, y=1000, width=70, height=750)

In [6]:
# instance_g_noise = instance.add_gaussian_noise(image_path= image_path, mean = 25, std=25)

In [7]:
# instance_shear = instance.shear(image_path=image_path, shear_x=-0.5, shear_y=0.5)

In [8]:
# instance_constrast = instance.adjust_contrast(image_path=image_path, alpha=1.5, beta=0)

In [12]:
# instance_scale = instance.scale(image_path= image_path, scale_factor= 0.25)